In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Embedding, SimpleRNN, LSTM

In [3]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import re

In [ ]:
with open('lastofus.txt', encoding='euckr') as f:
  line = f.readlines()

  for i in line:
    print(i)

In [ ]:
text = ''

array = []
with open('lastofus.txt', encoding='euckr') as f:
  lines = f.readlines()
  text = text.join([i for i in lines if re.match(r'^[A-Z].*:', i)])

text

In [ ]:
print(text)

In [6]:
arr = []
for t in text.split('\n')[:1001]:
  arr.append(t)

In [7]:
tokenizer = Tokenizer(lower=False, filters = '?!.,:;\'\"-%\n')
tokenizer.fit_on_texts(arr)

In [ ]:
tokenizer.word_index

In [10]:
len(tokenizer.word_index)

1311

In [ ]:
seq = tokenizer.texts_to_sequences(arr)
seq

In [ ]:
lengths = [len(line) for line in seq]
lengths

In [ ]:
plt.hist(lengths, bins=50)

In [14]:
seq = pad_sequences(seq, maxlen=12)
seq.shape

(1001, 12)

In [15]:
x = seq[:-1]
y = seq[1:]

In [16]:
x.shape

(1000, 12)

In [17]:
y.shape

(1000, 12)

In [18]:
x = x.reshape(1000, 12, 1)
y = y.reshape(1000, 12, 1)

In [19]:
from tensorflow.keras.utils import to_categorical

In [20]:
y_oh = to_categorical(y)
y_oh.shape

(1000, 12, 1312)

In [21]:
# x_oh = to_categorical(x)
# x_oh.shape

In [22]:
x.shape

(1000, 12, 1)

In [23]:
y_oh.shape

(1000, 12, 1312)

In [ ]:
model=Sequential()
model.add(Embedding(1312, 20))
model.add(LSTM(128, return_sequences=True))
model.add(Dropout(0.5))
model.add(LSTM(256, return_sequences=True))
model.add(Dropout(0.5))
model.add(Dense(1000, activation='softmax'))
model.add(Dropout(0.5))
model.add(Dense(1312, activation='softmax'))
model.summary()

In [25]:
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

In [ ]:
model.fit(x, y_oh, epochs=300, batch_size=20, validation_split=0.2)

In [27]:
model.save('lastofus_txt.h5')

In [41]:
pred = model.predict(x[30:31])
pred

array([[[7.4044055e-01, 1.5030812e-02, 1.1965322e-02, ...,
         4.4882876e-07, 4.5829611e-07, 4.5327096e-07],
        [7.2165900e-01, 1.6408185e-02, 1.2861840e-02, ...,
         5.4278939e-07, 5.5301911e-07, 5.4837648e-07],
        [6.7340553e-01, 1.9561367e-02, 1.5570279e-02, ...,
         7.2471431e-07, 7.3545823e-07, 7.3046385e-07],
        ...,
        [6.1767444e-02, 1.7619899e-02, 1.1183922e-02, ...,
         1.0493391e-06, 1.1005404e-06, 1.1368936e-06],
        [6.0469225e-02, 1.7262908e-02, 1.0937021e-02, ...,
         1.0460919e-06, 1.0985367e-06, 1.1350567e-06],
        [6.0466390e-02, 1.7262129e-02, 1.0936484e-02, ...,
         1.0460849e-06, 1.0985335e-06, 1.1350533e-06]]], dtype=float32)

In [42]:
pred.shape

(1, 12, 1312)

In [43]:
pred_tok = np.argmax(pred, axis=2)

In [44]:
pred_tok

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [ ]:
tokenizer.word_index

In [ ]:
idx_word = {}

for w in tokenizer.word_index:
  idx_word[tokenizer.word_index[w]] = w

idx_word

In [ ]:
pred_tok

In [47]:
input_x = x[30:31]

input_txt = [ t[0] for t in input_x[0]]

temp = ''

for t in input_txt:
  if t != 0:
    temp += idx_word[t]
    temp += ' '
temp

'Sarah Where the heck are you '

In [48]:
pred = model.predict(input_x)
pred_tok = np.argmax(pred, axis=2)

temp = ''

for l in pred_tok:
  for w in l:
    if w != 0:
      temp += idx_word[w]
      temp += ' '
temp
                      

''